# Основы RAG с Mistral AI

![](../../images/rag.png)

Генерация с расширенным поиском (RAG) — это AI-фреймворк, который объединяет возможности LLM и систем информационного поиска. Он полезен для ответов на вопросы или генерации контента с использованием внешних знаний. В RAG есть два основных этапа: 1) поиск: извлечение релевантной информации из базы знаний с помощью текстовых эмбеддингов, хранящихся в векторном хранилище; 2) генерация: вставка релевантной информации в промпт для LLM для генерации информации. В этом руководстве мы рассмотрим очень простой пример RAG с четырьмя реализациями:

- RAG с нуля с Mistral
- RAG с Mistral и LangChain
- RAG с Mistral и LlamaIndex
- RAG с Mistral и Haystack

## RAG с нуля

Этот раздел призван провести вас через процесс создания базового RAG с нуля. У нас две цели: во-первых, предложить пользователям всестороннее понимание внутренней работы RAG и демистифицировать основные механизмы; во-вторых, дать вам необходимые основы для создания RAG с минимальными зависимостями.

### Импорт необходимых пакетов
Первый шаг — установить необходимые пакеты `mistralai` и `faiss-cpu` и импортировать нужные пакеты:


In [ ]:
%pip install faiss-cpu==1.7.4 mistralai


In [ ]:
from mistralai import Mistral
import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key= getpass("Введите ваш API ключ")
client = Mistral(api_key=api_key)


### Получение данных

В этом очень простом примере мы получаем данные из эссе, написанного Полом Грэмом:


In [ ]:
response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text


Мы также можем сохранить эссе в локальный файл:


In [ ]:
f = open('essay.txt', 'w')
f.write(text)
f.close()


In [ ]:
len(text)


## Разделение документа на чанки

В системе RAG крайне важно разделить документ на более мелкие чанки, чтобы было эффективнее идентифицировать и извлекать наиболее релевантную информацию в процессе поиска позже. В этом примере мы просто разделяем текст по символам, объединяем 2048 символов в каждый чанк, и получаем 37 чанков.


In [ ]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]


In [ ]:
len(chunks)


#### Соображения:
- **Размер чанка**: В зависимости от вашего конкретного случая использования может потребоваться настроить или поэкспериментировать с различными размерами чанков и их перекрытием для достижения оптимальной производительности RAG. Например, меньшие чанки могут быть более полезными в процессах поиска, поскольку большие текстовые чанки часто содержат текст-заполнитель, который может затемнить семантическое представление. Таким образом, использование меньших текстовых чанков в процессе поиска может позволить системе RAG идентифицировать и извлекать релевантную информацию более эффективно и точно. Однако стоит учитывать компромиссы, связанные с использованием меньших чанков, такие как увеличение времени обработки и вычислительных ресурсов.

- **Как разделять**: Хотя самый простой метод — разделить текст по символам, есть и другие варианты в зависимости от случая использования и структуры документа. Например, чтобы избежать превышения лимитов токенов в вызовах API, может потребоваться разделить текст по токенам. Для поддержания связности чанков может быть полезно разделить текст по предложениям, абзацам или HTML-заголовкам. Если вы работаете с кодом, часто рекомендуется разделять по осмысленным фрагментам кода, например, используя парсер абстрактного синтаксического дерева (AST).

### Создание эмбеддингов для каждого текстового чанка

Для каждого текстового чанка нам затем нужно создать текстовые эмбеддинги, которые являются числовыми представлениями текста в векторном пространстве. Ожидается, что слова с похожими значениями будут находиться ближе друг к другу или иметь меньшее расстояние в векторном пространстве.

Чтобы создать эмбеддинг, используйте API эмбеддингов Mistral и модель эмбеддингов `mistral-embed`. Мы создаем функцию `get_text_embedding` для получения эмбеддинга из одного текстового чанка, а затем используем list comprehension для получения текстовых эмбеддингов для всех текстовых чанков.


In [ ]:
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings.create(
          model="mistral-embed",
          inputs=input
      )
    return embeddings_batch_response.data[0].embedding


In [ ]:
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])


In [ ]:
text_embeddings.shape


In [ ]:
text_embeddings


### Загрузка в векторную базу данных

После получения текстовых эмбеддингов обычной практикой является их хранение в векторной базе данных для эффективной обработки и поиска. Существует несколько векторных баз данных на выбор. В нашем простом примере мы используем векторную базу данных с открытым исходным кодом Faiss, которая позволяет эффективный поиск по сходству.

С Faiss мы создаем экземпляр класса Index, который определяет структуру индексирования векторной базы данных. Затем мы добавляем текстовые эмбеддинги в эту структуру индексирования.


In [ ]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)


#### Соображения:
- **Векторная база данных**: При выборе векторной базы данных следует учитывать несколько факторов, включая скорость, масштабируемость, облачное управление, расширенную фильтрацию и открытый исходный код против закрытого.

### Создание эмбеддингов для вопроса

Когда пользователи задают вопрос, нам также необходимо создать эмбеддинги для этого вопроса, используя те же модели эмбеддингов, что и раньше.


In [ ]:
question = "Какие две основные вещи автор делал до колледжа?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape


In [ ]:
question_embeddings


#### Соображения:
- **Гипотетические эмбеддинги документов (HyDE)**: В некоторых случаях вопрос пользователя может быть не самым релевантным запросом для определения релевантного контекста. Вместо этого может быть более эффективным сгенерировать гипотетический ответ или гипотетический документ на основе запроса пользователя и использовать эмбеддинги сгенерированного текста для поиска похожих текстовых чанков.

### Извлечение похожих чанков из векторной базы данных

Мы можем выполнить поиск в векторной базе данных с помощью `index.search`, которая принимает два аргумента: первый — это вектор эмбеддингов вопроса, а второй — количество похожих векторов для извлечения. Эта функция возвращает расстояния и индексы наиболее похожих векторов на вектор вопроса в векторной базе данных. Затем на основе возвращенных индексов мы можем извлечь фактические релевантные текстовые чанки, соответствующие этим индексам.


In [ ]:
D, I = index.search(question_embeddings, k=2)
print(I)


In [ ]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)


#### Соображения:
- **Методы поиска**: Существует много различных стратегий поиска. В нашем примере мы показываем простой поиск по сходству с эмбеддингами. Иногда, когда доступны метаданные для данных, лучше сначала отфильтровать данные на основе метаданных перед выполнением поиска по сходству. Также существуют другие статистические методы поиска, такие как TF-IDF и BM25, которые используют частоту и распределение терминов в документе для идентификации релевантных текстовых чанков.

- **Извлеченный документ**: Всегда ли мы извлекаем отдельный текстовый чанк как есть? Не всегда.
    - Иногда мы хотели бы включить больше контекста вокруг фактически извлеченного текстового чанка. Мы называем фактически извлеченный текстовый чанк "дочерним чанком", и наша цель — извлечь более крупный "родительский чанк", которому принадлежит "дочерний чанк".
    - Иногда мы также можем захотеть присвоить веса нашим извлеченным документам. Например, взвешенный по времени подход поможет нам извлечь самый последний документ.
    - Одна распространенная проблема в процессе поиска — это проблема "потери в середине", когда информация в середине длинного контекста теряется. Наши модели пытались смягчить эту проблему. Например, в задаче passkey наши модели продемонстрировали способность находить "иголку в стоге сена", извлекая случайно вставленный пароль в длинном промпте, до 32k длины контекста. Однако стоит рассмотреть возможность экспериментов с переупорядочиванием документа, чтобы определить, приведет ли размещение наиболее релевантных чанков в начале и конце к улучшенным результатам.

### Объединение контекста и вопроса в промпте и генерация ответа

Наконец, мы можем предложить извлеченные текстовые чанки в качестве контекстной информации в промпте. Вот шаблон промпта, где мы можем включить как извлеченный текст, так и вопрос пользователя в промпт.


In [ ]:
prompt = f"""
Контекстная информация приведена ниже.
---------------------
{retrieved_chunk}
---------------------
Учитывая контекстную информацию, а не предварительные знания, ответьте на запрос.
Запрос: {question}
Ответ:
"""


In [ ]:
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)


In [ ]:
run_mistral(prompt)


#### Соображения:
- **Техники промптинга**: Большинство техник промптинга также можно использовать при разработке системы RAG. Например, мы можем использовать few-shot обучение для руководства ответами модели, предоставляя несколько примеров. Кроме того, мы можем явно указать модели форматировать ответы определенным образом.

В следующих разделах мы покажем вам, как сделать аналогичный базовый RAG с некоторыми популярными RAG-фреймворками.


## LangChain


In [ ]:
%pip install langchain langchain-mistralai langchain_community mistralai==0.4.2


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

# Загрузка данных
loader = TextLoader("essay.txt")
docs = loader.load()

# Разделение текста на чанки
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

# Определение модели эмбеддингов
embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=api_key)

# Создание векторного хранилища
vector = FAISS.from_documents(documents, embeddings)

# Определение интерфейса ретривера
retriever = vector.as_retriever()

# Определение LLM
model = ChatMistralAI(mistral_api_key=api_key)

# Определение шаблона промпта
prompt = ChatPromptTemplate.from_template("""Ответьте на следующий вопрос только на основе предоставленного контекста:

<context>
{context}
</context>

Вопрос: {input}""")

# Создание цепочки поиска для ответов на вопросы
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "Какие две основные вещи автор делал до колледжа?"})
print(response["answer"])


## LlamaIndex


In [ ]:
%pip install llama-index==0.10.55 llama-index-llms-mistralai==0.1.18 llama-index-embeddings-mistralai mistralai==0.4.2


In [ ]:
import os
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding

# Загрузка данных
reader = SimpleDirectoryReader(input_files=["essay.txt"])
documents = reader.load_data()

# Определение LLM и модели эмбеддингов
Settings.llm = MistralAI(model="mistral-medium", api_key=api_key)
Settings.embed_model = MistralAIEmbedding(model_name='mistral-embed', api_key=api_key)

# Создание индекса векторного хранилища
index = VectorStoreIndex.from_documents(documents)

# Создание движка запросов
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query(
    "Какие две основные вещи автор делал до колледжа?"
)
print(str(response))


## Haystack


In [ ]:
%pip install mistral-haystack==0.0.1 mistralai==0.4.2


In [ ]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.dataclasses import ChatMessage
from haystack.utils.auth import Secret

from haystack.components.builders import DynamicChatPromptBuilder
from haystack.components.converters import TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.writers import DocumentWriter
from haystack_integrations.components.embedders.mistral import MistralDocumentEmbedder, MistralTextEmbedder
from haystack_integrations.components.generators.mistral import MistralChatGenerator

document_store = InMemoryDocumentStore()

docs = TextFileToDocument().run(sources=["essay.txt"])
split_docs = DocumentSplitter(split_by="passage", split_length=2).run(documents=docs["documents"])
embeddings = MistralDocumentEmbedder(api_key=Secret.from_token(api_key)).run(documents=split_docs["documents"])
DocumentWriter(document_store=document_store).run(documents=embeddings["documents"])

text_embedder = MistralTextEmbedder(api_key=Secret.from_token(api_key))
retriever = InMemoryEmbeddingRetriever(document_store=document_store)
prompt_builder = DynamicChatPromptBuilder(runtime_variables=["documents"])
llm = MistralChatGenerator(api_key=Secret.from_token(api_key), model='mistral-small')

chat_template = """Ответьте на следующий вопрос на основе содержимого документов.\n
                Вопрос: {{query}}\n
                Документы:
                {% for document in documents %}
                    {{document.content}}
                {% endfor%}
                """
messages = [ChatMessage.from_user(chat_template)]

rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", text_embedder)
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", llm)

rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever.documents", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder.prompt", "llm.messages")

question = "Какие две основные вещи автор делал до колледжа?"

result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"template_variables": {"query": question}, "prompt_source": messages},
        "llm": {"generation_kwargs": {"max_tokens": 225}},
    }
)

print(result["llm"]["replies"][0].content)


## Заключение

Это руководство продемонстрировало, как создать базовую систему RAG четырьмя различными способами:
1. **С нуля** используя Faiss для векторной базы данных
2. **С LangChain** для более структурированного подхода с цепочками
3. **С LlamaIndex** для простого подхода с индексами
4. **С Haystack** для пайплайн-ориентированного подхода

Каждый подход имеет свои преимущества и может быть выбран в зависимости от ваших конкретных потребностей и предпочтений в архитектуре приложения.
